### ECE-GY 6123
### Nikita Makarov, Fall 2024

### Homework 3

Problem 1a:

In [5]:
import numpy as np
import cv2

# Create Gaussian Pyramid function
def gaussian_pyramid(im_data: np.ndarray, J: int) -> list[np.ndarray]:
    '''
    Compute and return the J-level Gaussian pyramid decomposition of a grayscale image.

    Parameters:
    im_data: 2D numpy array of the image data in grayscale
    J: int of the Gaussian pyramid level

    Returns:
    G: list of size J filled with 2D numpy arrays of the Gaussian pyramid, with the first element being level J
    '''

    # Create Gaussian pyramid structure
    G = [im_data] # Create list with first element as level J

    return G

image = cv2.imread('coins.png', cv2.IMREAD_GRAYSCALE)
pass